In [9]:
# More imports
import tensorflow as tf
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.layers import Input, Conv1D, Dense, Flatten,GRU, Dropout, MaxPooling1D
from tensorflow.keras.models import Model
from keras.callbacks import EarlyStopping
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

In [10]:
!git clone https://github.com/baontn/Bremen-Big-data-2019.git

In [11]:
def read_files_dummy(obj):
    import os
    obj = obj
    length = []
    paths = f"Bremen-Big-data-2019/movement/Subject{obj}" 
    dirs = os.listdir(paths)
    dirs.sort()
    for files in dirs:
        paths = f"Bremen-Big-data-2019/movement/Subject{obj}/{files}"
        train = pd.read_csv(paths)
        gi = len(train)
        length.append(gi)
    return length

In [12]:
    def read_files_train(obj):
        import os
        from sklearn.preprocessing import StandardScaler
        obj = obj
        h = 4000
        dimen = 19
        X = []
        scaler = StandardScaler()      
        
        paths = f"Bremen-Big-data-2019/movement/Subject{obj}" #note the list of training subjects
        dirs = os.listdir(paths)
        dirs.sort()
        for files in dirs:
            train_try =  np.zeros((h, dimen))
            paths = f"Bremen-Big-data-2019/movement/Subject{obj}/{files}"
            train = pd.read_csv(paths)
            train = scaler.fit_transform(train)
            if len(train) <= h:
                a = h - len(train)
                for i in range(len(train)):
                    train_try[a+i] = np.array(train)[i:i+1][0]             
                X.append(train_try)
            else:
                for i in range(0, h):
                    train_try[i] = np.array(train)[i:i+1][0]          
                X.append(train_try)                    
        return X, scaler

In [13]:
    def read_files_test(obj, scaler):
        import os
        obj = obj
        h = 4000
        dimen = 19
        X = []      
        
        paths = f"Bremen-Big-data-2019/movement/Subject{obj}" 
        dirs = os.listdir(paths)
        dirs.sort()
        for files in dirs:
            train_try =  np.zeros((h, dimen))
            paths = f"Bremen-Big-data-2019/movement/Subject{obj}/{files}"
            train = pd.read_csv(paths)
            train = scaler.transform(train)
            if len(train) <= h:
                a = h - len(train)
                for i in range(len(train)):
                    train_try[a+i] = np.array(train)[i:i+1][0]          # neu train < h thi t pad 0 o dang truoc roi lap data vo sau. 
                X.append(train_try)
            else:
                for i in range (0,h):
                    train_try[i] = np.array(train)[i:i+1][0]          
                X.append(train_try)  
        return X

In [14]:
    def read_files_dummy2(year):
        import os
        year = year
        h = 3000
        dimen = 19
        X = []
        paths = f"Bremen-Big-data-2019/movement/Subject{year}" #note the list of training subjects
        dirs = os.listdir(paths)
        dirs.sort()
        for files in dirs:
            train_try =  np.zeros((h, dimen))
            paths = f"Bremen-Big-data-2019/movement/Subject{year}/{files}"
            train = pd.read_csv(paths)
            if len(train) <= h:
                a = h - len(train)
                for i in range(len(train)):
                    train_try[a+i] = np.array(train)[i:i+1][0]             
                X.append(train_try)
            else:
                for i in range(0, h):
                    train_try[i] = np.array(train)[i:i+1][0]          
                X.append(train_try)                    
        return X

In [15]:
# Plot confusion matrix
from sklearn.metrics import confusion_matrix
import itertools

def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
  """
  This function prints and plots the confusion matrix.
  Normalization can be applied by setting `normalize=True`.
  """
  plt.rcParams["figure.figsize"] = (30,10)
  if normalize:
      cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
      print("Normalized confusion matrix")
  else:
      print('Confusion matrix, without normalization')

  print(cm)

  plt.imshow(cm, interpolation='nearest', cmap=cmap)
  plt.title(title)
  plt.colorbar()
  tick_marks = np.arange(len(classes))
  plt.xticks(tick_marks, classes, rotation=45)
  plt.yticks(tick_marks, classes)

  fmt = '.2f' if normalize else 'd'
  thresh = cm.max() / 2.
  for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
      plt.text(j, i, format(cm[i, j], fmt),
               horizontalalignment="center",
               color="white" if cm[i, j] > thresh else "black")

  plt.tight_layout()
  plt.ylabel('True label')
  plt.xlabel('Predicted label')
  plt.show()

# Data Exploration

In [16]:
obj_list = ['02','03','04','05','06','07','08','09',11,12,13,16,17,18,19]
X_1 = []
for obj in obj_list:
    X = read_files_dummy(obj)
    #X = np.array(X)
    X_1.append(X)
    print('done')

In [17]:
num_bins = 10
n, bins, patches = plt.hist(X_1, num_bins, facecolor='blue', alpha=0.1)
plt.show()

In [18]:
x = read_files_dummy2(14)

In [19]:
k = []
for i in range(len(x)):
    for j in range(3000):
        f = x[i][j]
        m = 0
        for l in f:
            m = m+l
        if m !=0:
            for n in f:
                k.append(n)

In [20]:
num_bins = 50
n, bins, patches = plt.hist(k, num_bins, facecolor='blue', alpha=0.1)
plt.show()

In [21]:
np.array(x).shape

In [23]:
obj_list = ['02','03','04','05','06','07','08','09',11,12,13,16]
X_1 = []
for obj in obj_list:
    X = read_files_train(obj)[0]
    X = np.array(X)
    X_1.append(X)
    print('done')
    if obj == 16:
        scaler = read_files_train(obj)[1]

In [24]:
from sklearn.preprocessing import LabelEncoder
y_train = pd.read_csv('Bremen-Big-data-2019/movement/train.csv')
encode = LabelEncoder()
y = encode.fit_transform(y_train.Label)

In [25]:
exclusion_list = []
for x in range(len(y)):
    if y[x] == 10:
        exclusion_list.append(x)

In [26]:
rep = np.ones((4000,19)) 

In [27]:
for i in exclusion_list:
    j = i-1756     #
    X_1[4][j] = rep

In [28]:
x_train = np.array(X_1[0])
for i in X_1[1:]:
  xc = np.append(x_train,i, axis = 0)
  x_train = xc

In [29]:
obj_list = [17,18,19] 
X_1 = []
for obj in obj_list:
    X = read_files_test(obj, scaler)
    X = np.array(X)
    X_1.append(X)
    print('done')

In [30]:
x_test = np.array(X_1[0])
for i in X_1[1:]:
  xc = np.append(x_test,i, axis = 0)
  x_test = xc

In [31]:
split_index = x_train.shape[0]
split_index

In [32]:
x_test.shape

In [33]:
y_train = y[:split_index]

In [34]:
y_test = y[split_index:]

In [35]:
pd.Series(y_train).value_counts().plot(kind='bar')

In [36]:
obj_list = ['01',10,14,15] 
X_1 = []
for obj in obj_list:
    X = read_files_test(obj, scaler)
    X = np.array(X)
    X_1.append(X)
    print('done')

In [37]:
x_pred = np.array(X_1[0])
for i in X_1[1:]:
  xc = np.append(x_pred,i, axis = 0)
  x_pred = xc

In [38]:
label = np.unique(y)
label = encode.inverse_transform(label)
label #lay la cai fake data ma organizer cai vo

In [39]:
# Build the model using the functional API
i = Input(shape=x_train[0].shape)
x = Conv1D(256, (3), strides=2, activation='relu')(i)
x = Conv1D(256, (3), strides=2, activation='relu')(x)
x = MaxPooling1D((2))(x)
x = Conv1D(256, (3), strides=2, activation='relu')(x)
x = Conv1D(256, (3), strides=2, activation='relu')(x)
x = MaxPooling1D((2))(x)
x = Conv1D(512, (3), strides=1, activation='relu')(x)
x = Conv1D(512, (3), strides=1, activation='relu')(x)
x = MaxPooling1D((2))(x)
x = Conv1D(1024, (3), strides=1, activation='relu')(x)
x = Conv1D(1024, (3), strides=1, activation='relu')(x)
x = MaxPooling1D((2))(x)
x = Conv1D(1024, (6), strides=1, activation='relu')(x)
x = Conv1D(1024, (6), strides=1, activation='relu')(x)
x = MaxPooling1D((2))(x)
x = GRU(2048)(x)
x = Flatten()(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.2)(x)
x = Dense(23, activation='softmax')(x)

model = Model(i, x)

In [40]:
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=0.0085,
    decay_steps=10000,
    decay_rate=0.9)
optimizer = tf.keras.optimizers.SGD(learning_rate=lr_schedule)

In [41]:
# Compile the model
model.compile(optimizer=optimizer,
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'] )

In [42]:
r = model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=13,batch_size = 1)

In [43]:
# Plot loss per iteration
import matplotlib.pyplot as plt
plt.plot(r.history['loss'], label='loss')
plt.plot(r.history['val_loss'], label='val_loss')
plt.legend()

In [44]:
# Plot accuracy per iteration
plt.plot(r.history['accuracy'], label='acc')
plt.plot(r.history['val_accuracy'], label='val_acc')
plt.legend()

In [45]:
# Evaluate the model
print(model.evaluate(x_test, y_test))

In [46]:
p_test = model.predict(x_test).argmax(axis=1)
cm = confusion_matrix(y_test, p_test)
plot_confusion_matrix(cm, list(range(23)))


In [47]:
x_pred.shape

In [48]:
y_pred = model.predict(x_pred).argmax(axis=1)

In [49]:
pd.Series(y_pred).value_counts().plot(kind='bar')

In [50]:
#from sklearn.preprocessing import LabelEncoder
y_data = pd.read_csv('Bremen-Big-data-2019/movement/challenge_labels.csv')
#encode = LabelEncoder()
y_true = encode.transform(y_data.Label)

In [51]:
a = 0
for i in range(len(y_true)):  
    if y_true[i] == y_pred[i]:
        a+=1
    else:
        pass
a/1738    

In [52]:
a

In [53]:
cm = confusion_matrix(y_true, y_pred)
plot_confusion_matrix(cm, list(range(23)))

In [54]:
df = pd.DataFrame()
df['Pred_1'] = y_pred

In [55]:
df.to_csv('pred1.csv')

In [56]:
import os
os.getcwd()